<a href="https://colab.research.google.com/github/shivanswamynathan/langchain/blob/main/LLM_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


HuggingFace

In [2]:
import os
from langchain import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""

In [3]:
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.6,"max_length":64})

<ipython-input-3-bd01062b9a7c>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.6,"max_length":64})


In [6]:
llm_huggingface.predict("Can you tell me the capital of Russia")


'moscow'

In [5]:
llm_huggingface.predict("who is current prime minister of india")

'Narendra Modi'

Prompt template

In [39]:
from langchain import PromptTemplate,LLMChain

country_template=PromptTemplate(
    input_variables=["country"],
    template=" tell me the capital of {country}"
)




In [40]:
country_template.format(country="india")

' tell me the capital of india'

Chain

In [41]:
chain1 = LLMChain(llm=llm_huggingface, prompt=country_template)

In [42]:
print(chain1.run("india"))

Delhi


In [43]:
from langchain import PromptTemplate,LLMChain

promt_template=PromptTemplate(
    input_variables=["country"],
    template=" tell me the prime minister of {country}"
)

In [44]:
chain2 = LLMChain(llm=llm_huggingface, prompt=promt_template)

In [45]:
print(chain2.run("india"))

Narendra Modi


combining multiple chains using simple sequential chain

In [46]:
from langchain.chains import SimpleSequentialChain

CM_template=PromptTemplate(
    input_variables=["capital"],
    template=" tell me the Chief Minister of {capital}"
)

chain3 = LLMChain(llm=llm_huggingface, prompt=CM_template)

In [47]:

chain = SimpleSequentialChain(chains=[chain1,chain3])

print(chain.run("india"))

Arun Jaitley


sequential chain

In [48]:
from langchain.chains import SequentialChain

country_template=PromptTemplate(
    input_variables=["country"],
    template=" tell me the capital of {country}"
)

chain1 = LLMChain(llm = llm_huggingface,prompt = country_template,output_key="capital")

In [49]:
CM_template=PromptTemplate(
    input_variables=["capital"],
    template=" tell me the Chief Minister of {capital}"
)

chain3 = LLMChain(llm=llm_huggingface, prompt=CM_template,output_key="CM")

In [56]:
sequence_chain = SequentialChain(chains = [chain1,chain3],input_variables = ["country"],output_variables = ["capital","CM"])
print(sequence_chain({"country" : "india"}))

{'country': 'india', 'capital': 'Delhi', 'CM': 'Arun Jaitley'}


Chatmodels With Gemini Pro

In [57]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00


In [60]:
os.environ["GOOGLE_API_KEY"] = ""

In [61]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [65]:
chatllm = ChatGoogleGenerativeAI(api_key = os.environ["GOOGLE_API_KEY"] , temperature=0.6 ,model="gemini-1.5-flash")

In [68]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])


AIMessage(content='* I told my AI assistant I needed a joke about AI.  It said, "I\'m still processing...  ...  ...  ...  Is this thing on?"\n\n* Why was the AI so bad at poker?  Because it kept bluffing with a completely transparent algorithm!\n\n* What\'s an AI\'s favorite type of music?  Chip-tune!  (Get it?  Because... chips...  Never mind.)\n\n* I tried to explain a joke to my AI, but it just kept asking for more data.  I think it\'s trying to *learn* humor, but it\'s currently stuck in the "data acquisition" phase of its development.  It\'s like a toddler who only laughs at farts.  A *very* data-driven toddler.\n\n* My AI woke me up this morning by saying, "Good morning!  Your probability of having a productive day is 73.2%. Please adjust your caffeine intake accordingly."  I\'m pretty sure it\'s judging me.  And it has *statistics* to back it up.  The horror!\n\n* Why did the AI cross the road?  Because it detected a statistically significant increase in pedestrian traffic on th

Prompt Template + LLM +Output Parsers

In [69]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [70]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [71]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [81]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [83]:
chain.invoke({"text":"curriculum"}).content # or we can use chain.invoke({"text":"curriculum"}).content

['Syllabus', ' course of study', ' program', ' curriculum', ' schedule']